In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('2013_Salaries__Pennsylvania_State_System_of_Higher_Education.csv', index_col=0)
df

,Name,Base Pay,University or Office,Position
index,,,,
0,"Driscoll, Michael Allan",$275000,Indiana,University President
1,"Weisenstein, Greg R",$241935,West Chester,University President
2,"Welsh, Marcia L",$230000,East Stroudsburg,University President
3,"Norton, Cheryl J.",$225000,Slippery Rock,University President
4,"Wollman, Julie E.",$225000,Edinboro,University President
...,...,...,...,...
12627,"Black, Jacqueline D",$1254,Edinboro,"Instructor, PT Temp"
12628,"Webster, Kim",$1245,Kutztown,Instructor
12629,"Hanelly, William T",$152950,Lock Haven,Vice President/Fin.Admin


In [5]:
df.Position.unique()

array(['University President', 'Provost & VP Academic Affairs',
       'Provost & VP for Academic Affairs', ...,
       'Vice President/Fin.Admin',
       'Vice Chancellor Academic & Student Affairs', 'Chancellor'],
      shape=(3315,), dtype=object)